In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
# import numpy as np

In [ ]:
database_type = "/training"

#### Interpolating Data across patient timeseries

In [ ]:
finalMerge = pd.read_csv('../eICU' + database_type + '/finalMerge.csv')

In [ ]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))
finalData = pd.DataFrame()

In [ ]:
for id in idList:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'diagnosis'] = df['diagnosis'].interpolate(method='linear')
    df['diagnosis'].fillna('0', inplace=True)
    finalData = finalData.append(df)
    

In [ ]:
path = "../../eICU/training/"
finalData.to_csv(path + "finalData.csv", sep=',', index=False, encoding='utf-8')